In [73]:
import json
import re

def get_global_premises(constants):
    return {d.split(':')[0].strip():":".join(d.split(':')[1:]) for d in constants}

def get_tactic_premises(premises, tac):
    tokens = set(re.findall(r'\b\w+\b', tac))
    return [word for word in premises.keys() if word in tokens and len(word) > 1]

def print_trace(data):
    premises = get_global_premises(data['constants'])
    for g_before, step in zip(data['goals'], data['evaluation']):
        tac = step['tactic']
        p = get_tactic_premises(premises, tac)

        print("<goals>")
        for i, g in enumerate(g_before):
            print(f"Goal {i}")
            print(g)
        print("</goals>\n")

        if p:
            print(f"SEARCH {" ".join(p)}\n")
            
            print("<result>")
            for p in p:
                print(f"{p}: {premises[p]}\n")
            print("</result>")

        print("<tactic>")
        print(step['tactic'])
        print("</tactics>\n")



In [74]:
with open('../dataset/traced_mathcomp/galois/term_81.json', 'r') as f:
    # Load the JSON data from the file
    data = json.load(f)

print_trace(data)


<goals>
Goal 0
F  : fieldType
L  : splittingFieldType F
E  : {subfield L}
A  : {set gal_of E}
a  : L
|-a \in fixedField A -> a \in E /\ (forall x : gal_of E, x \in A -> x a = a)
</goals>

SEARCH fixedFieldP memv_capP

<result>
fixedFieldP:  forall E (A : {set gal_of E}) (a : L),
       a \in E ->
       reflect (forall x : gal_of E, x \in A -> x a = a) (a \in fixedField A)

memv_capP:  reflect (?w \in ?U /\ ?w \in ?V) (?w \in (?U :&: ?V)%VS)
where
?K : [F : fieldType  L : splittingFieldType F |- fieldType]
?vT : [F : fieldType  L : splittingFieldType F |- vectType ?K]
?w : [F : fieldType  L : splittingFieldType F |- Vector.sort ?vT]
?U : [F : fieldType  L : splittingFieldType F |- {vspace ?vT}]
?V : [F : fieldType  L : splittingFieldType F |- {vspace ?vT}]

</result>
<tactic>
by move=> fixAa; have [Ea _] := memv_capP fixAa; have:= fixedFieldP Ea fixAa.
</tactics>



In [75]:
with open('../dataset/traced_mathcomp/poly/term_15.json', 'r') as f:
    # Load the JSON data from the file
    data = json.load(f)

print_trace(data)



<goals>
Goal 0
R  : semiRingType
s  : seq R
|-size (Poly s) <= size s
</goals>

SEARCH polyseqC eqxx

<result>
polyseqC:  forall c, c%:P = nseq (c != 0) c

eqxx:  forall (T : eqType) (x : T), x == x

</result>
<tactic>
elim: s => [|c s IHs] /=; first by rewrite polyseqC eqxx.
</tactics>

<goals>
Goal 0
R  : semiRingType
c  : R
s  : seq R
IHs  : size (Poly s) <= size s
|-size (cons_poly c (Poly s)) <= (size s).+1
</goals>

SEARCH size_polyC ifP polyseq_cons

<result>
size_polyC:  forall c, size c%:P = (c != 0)

ifP:  forall (A : Type) (b : bool) (vT vF : A),
       if_spec b vT vF (b = false) b (if b then vT else vF)

polyseq_cons:  forall c p, cons_poly c p = (if ~~ nilp p then c :: p else c%:P)

</result>
<tactic>
by rewrite polyseq_cons; case: ifP => // _; rewrite size_polyC; case: (~~ _).
</tactics>



{'fixedFieldP': ' forall E (A : {set gal_of E}) (a : L),\n       a \\in E ->\n       reflect (forall x : gal_of E, x \\in A -> x a = a) (a \\in fixedField A)',
 'fixedField': ' forall V, {set gal_of V} -> {vspace L}',
 'memv_capP': ' reflect (?w \\in ?U /\\ ?w \\in ?V) (?w \\in (?U :&: ?V)%VS)\nwhere\n?K : [F : fieldType  L : splittingFieldType F |- fieldType]\n?vT : [F : fieldType  L : splittingFieldType F |- vectType ?K]\n?w : [F : fieldType  L : splittingFieldType F |- Vector.sort ?vT]\n?U : [F : fieldType  L : splittingFieldType F |- {vspace ?vT}]\n?V : [F : fieldType  L : splittingFieldType F |- {vspace ?vT}]',
 'gal_of': ' {vspace L} -> Type'}